In [1]:
import numpy as np
from copy import deepcopy
from src.sim.scenario import Scenario
from src.logging import get_logger

logger = get_logger(__name__)

%load_ext autoreload
%autoreload 2

In [2]:
# Generate markets and prices
market_name = "wbtc"
scenario = Scenario("baseline", market_name)
print(
    f"Running {scenario.num_steps} steps with frequency {scenario.pricepaths.config['freq']}."
)

[INFO][17:05:29][src.sim.scenario]-62943: Fetching sim_market from subgraph.
[INFO][17:05:35][src.sim.scenario]-62943: Market snapshot as 2023-12-26 16:26:11
[INFO][17:05:35][src.sim.scenario]-62943: Bands snapshot as 2023-12-25 21:27:35
[INFO][17:05:35][src.sim.scenario]-62943: Users snapshot as 2023-12-26 13:17:47
[INFO][17:05:38][src.utils.poolgraph]-62943: Found 20 valid trading cycles of length 3.


Running 168 steps with frequency 1h.


In [3]:
METADATA = scenario.llamma.metadata


def approx_equal(a, b, tol=1e-6):
    """Returns True if a and b are approximately equal."""
    if a == 0:
        return b == 0
    return abs(a - b) / a < tol


def find_state(user):
    for user_state in METADATA["userStates"]:
        if user_state["user"]["id"] == user:
            return user_state
    return None


llamma = scenario.llamma
controller = scenario.controller

In [4]:
# Check bands were correctly initialized
for band in llamma.metadata["bands"]:
    i = int(band["index"])
    x = llamma.bands_x[i] / 1e18
    y = llamma.bands_y[i] / 1e18
    assert approx_equal(float(band["stableCoin"]), x), i
    assert approx_equal(float(band["collateral"]), y), i
    assert approx_equal(
        float(band["priceOracleUp"]), llamma.p_oracle_up(i) / 1e18, tol=1e-3
    ), i
    assert approx_equal(
        float(band["priceOracleDown"]), llamma.p_oracle_down(i) / 1e18, tol=1e-3
    ), i
    if x > 0 and y > 0:
        assert i == llamma.active_band
        logger.info("Active band is %d", i)

[INFO][17:05:38][__main__]-62943: Active band is -33


In [5]:
to_liquidate = controller.users_to_liquidate()
users_to_liquidate = [user.user for user in to_liquidate]
logger.info("%d users were loaded underwater.", len(to_liquidate))

[INFO][17:05:38][__main__]-62943: 23 users were loaded underwater.


In [6]:
# Compare loaded users to expected users
good_users = 0
for user_state in llamma.metadata["userStates"]:
    user = user_state["user"]["id"]
    assert controller.loan_exists(user)
    loaded_state = controller.user_state(user)
    collateral, stablecoin, debt, N = loaded_state
    success = approx_equal(
        float(user_state["health"]), controller.health(user) / 1e18, tol=0.01
    )
    if float(user_state["health"]) < 0:
        assert controller.health(user) < 0
        if controller.health(user, full=True) < 0:
            assert user in users_to_liquidate
    good_users += success

logger.info(
    "%d out of %d user healths matched snapshot to 2dps.",
    good_users,
    len(controller.loan),
)

[INFO][17:05:38][__main__]-62943: 48 out of 259 user healths matched snapshot to 2dps.


In [7]:
# Test that user states were correctly loaded into LLAMMA
for tick in range(llamma.min_band, llamma.max_band + 1):
    total = 0
    total_y = 0
    total_x = 0
    at_least_one_user = False
    for user, shares in llamma.user_shares.items():
        dep = float(find_state(user)["collateralUp"]) * 1e18
        assert approx_equal(sum(shares.ticks), dep, 0.01), (sum(shares.ticks), dep)
        n1, n2 = shares.n1, shares.n2
        if n1 <= tick <= n2:
            i = tick - n1
            total += shares.ticks[i]
            xy = llamma._get_xy(user, False)
            total_y += xy[1][i]
            total_x += xy[0][i]
            at_least_one_user = True
    if at_least_one_user:
        assert approx_equal(
            llamma.bands_y[tick], total_y * llamma.COLLATERAL_PRECISION, 0.01
        )  # rounding errors
        assert approx_equal(llamma.bands_x[tick], total_x)
        assert approx_equal(llamma.total_shares[tick], total)

In [8]:
# ts = scenario.price_oracle._block_timestamp + 60*60
# sample = 1_000_000
# _p = int(
#     sample
#     * 10**18
# )
# scenario.price_oracle.set_price(_p)
# scenario._increment_timestamp(ts)
# scenario.llamma._price_oracle_w()

to_liquidate = controller.users_to_liquidate()
users_to_liquidate = [user.user for user in to_liquidate]
logger.info("%d users were loaded underwater.", len(to_liquidate))

# Test that only a small portion of debt is liquidatable at start
damage = 0
for pos in to_liquidate:
    damage += pos.debt
    state = find_state(pos.user)
    logger.info(
        "Liquidating %s: n1: %d, n2: %d, original health: %f, debt: %f, lossPct: %f",
        pos.user,
        int(state["n1"]),
        int(state["n2"]),
        float(state["health"]),
        float(state["debt"]),
        float(state["lossPct"]),
    )
logger.info(
    "%.2f%% of debt was incorrectly loaded with sub-zero health (%d crvUSD)",
    round(damage / controller.total_debt() * 100, 2),
    damage / 1e18,
)

[INFO][17:05:40][__main__]-62943: 23 users were loaded underwater.
[INFO][17:05:40][__main__]-62943: Liquidating 0xec263efe3df2b0a5148df59b2551cf46ce8c763e: n1: -27, n2: -24, original health: 0.041001, debt: 125998.166012, lossPct: 0.000002
[INFO][17:05:40][__main__]-62943: Liquidating 0xe3cf28f4c11b6ef4313e7c23df526f75bab0bc64: n1: -23, n2: -20, original health: 0.040781, debt: 25286.610226, lossPct: 3.408796
[INFO][17:05:40][__main__]-62943: Liquidating 0xce1a34fccb14d13f8f6b6dcb0f3df2d140618c16: n1: -30, n2: -21, original health: 0.013698, debt: 120698.969825, lossPct: 2.336679
[INFO][17:05:40][__main__]-62943: Liquidating 0xbb482f13749eccabdaa3f923e133c9abcbbdfab3: n1: -27, n2: -18, original health: 0.041755, debt: 34819.026542, lossPct: 2.764514
[INFO][17:05:40][__main__]-62943: Liquidating 0xae5c67ceb16b4851f169ec1c65f405d7e6308b90: n1: -33, n2: -18, original health: 0.038092, debt: 300558.595163, lossPct: 0.742809
[INFO][17:05:40][__main__]-62943: Liquidating 0xa05fe74fb4fbb6c0d

In [14]:
ts = scenario.price_oracle._block_timestamp + 60 * 60
sample = 42371
_p = int(sample * 10**18)
scenario.price_oracle.set_price(_p)
scenario._increment_timestamp(ts)
scenario.llamma._price_oracle_w()

to_liquidate = controller.users_to_liquidate()
users_to_liquidate = [user.user for user in to_liquidate]
logger.info("%d users were loaded underwater.", len(to_liquidate))

# Test that only a small portion of debt is liquidatable at start
damage = 0
for pos in to_liquidate:
    damage += pos.debt
    state = find_state(pos.user)
    logger.info(
        "Liquidating %s: n1: %d, n2: %d, original health: %f, debt: %f, lossPct: %f",
        pos.user,
        int(state["n1"]),
        int(state["n2"]),
        float(state["health"]),
        float(state["debt"]),
        float(state["lossPct"]),
    )
logger.info(
    "%.2f%% of debt was incorrectly loaded with sub-zero health (%d crvUSD)",
    round(damage / controller.total_debt() * 100, 2),
    damage / 1e18,
)

[INFO][17:10:25][__main__]-62943: 14 users were loaded underwater.
[INFO][17:10:25][__main__]-62943: Liquidating 0xe3cf28f4c11b6ef4313e7c23df526f75bab0bc64: n1: -23, n2: -20, original health: 0.040781, debt: 25286.610226, lossPct: 3.408796
[INFO][17:10:25][__main__]-62943: Liquidating 0xce1a34fccb14d13f8f6b6dcb0f3df2d140618c16: n1: -30, n2: -21, original health: 0.013698, debt: 120698.969825, lossPct: 2.336679
[INFO][17:10:25][__main__]-62943: Liquidating 0xbb482f13749eccabdaa3f923e133c9abcbbdfab3: n1: -27, n2: -18, original health: 0.041755, debt: 34819.026542, lossPct: 2.764514
[INFO][17:10:25][__main__]-62943: Liquidating 0xae5c67ceb16b4851f169ec1c65f405d7e6308b90: n1: -33, n2: -18, original health: 0.038092, debt: 300558.595163, lossPct: 0.742809
[INFO][17:10:25][__main__]-62943: Liquidating 0xa05fe74fb4fbb6c0dfb213c8af532286f473e0c4: n1: -27, n2: -24, original health: 0.036117, debt: 34884.171266, lossPct: 3.877215
[INFO][17:10:25][__main__]-62943: Liquidating 0x9b193f070757bb02cd

In [15]:
llamma.price_oracle() / 1e18

42374.61575764005

In [9]:
suspect = "0x84f8f8dcdcd68afc6659b54454b428721e95df2e"
llamma.get_xy(suspect)

[[0, 0, 0, 0], [23143122, 12388167, 12339377, 11703288]]

In [19]:
llamma.bands_y[-28] / 1e18

10.600326074308244

In [9]:
llamma.get_p() / 1e18

39191.27588540292

In [10]:
scenario.prepare_for_run()

[INFO][16:25:48][src.sim.scenario]-61933: Equilibrated prices with 4 arbitrages with total profit 91159
[WARNING][16:25:48][src.sim.scenario]-61933: 22 users were loaded underwater.
[INFO][16:25:48][src.sim.scenario]-61933: Liquidating 0xec263efe3df2b0a5148df59b2551cf46ce8c763e: with debt 125998166012188247746558.
[INFO][16:25:48][src.sim.scenario]-61933: Liquidating 0xe3cf28f4c11b6ef4313e7c23df526f75bab0bc64: with debt 25286610225724004201114.
[INFO][16:25:48][src.sim.scenario]-61933: Liquidating 0xce1a34fccb14d13f8f6b6dcb0f3df2d140618c16: with debt 120698969824681607785994.
[INFO][16:25:48][src.sim.scenario]-61933: Liquidating 0xbb482f13749eccabdaa3f923e133c9abcbbdfab3: with debt 34819026542240597071858.
[INFO][16:25:48][src.sim.scenario]-61933: Liquidating 0xae5c67ceb16b4851f169ec1c65f405d7e6308b90: with debt 300558595163070908617553.
[INFO][16:25:48][src.sim.scenario]-61933: Liquidating 0xa05fe74fb4fbb6c0dfb213c8af532286f473e0c4: with debt 34884171266382455395327.
[INFO][16:25:48][

In [21]:
llamma.p_oracle_up(-30) / 1e18

42254.82875562195

In [35]:
controller.health("0xec263efe3df2b0a5148df59b2551cf46ce8c763e", full=True) / 1e18

24.1472326147092

In [33]:
llamma.price_oracle()

42064902914593603452928